# Agents

* Custom Tools

In [6]:
import os
import warnings
from datetime import datetime
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.agents import AgentType, initialize_agent, load_agent, Tool, tool, load_tools

# Config
load_dotenv(".env")
warnings.filterwarnings("ignore")
api_key = os.environ.get("key")

```bash
LLM ---> Tools DocStrings(Look for tools) ---> Pick Tool (select tools) --
                                                                          |
                                             Observation <--  Action   <--
```

In [2]:
@tool
def coolest_community(text: str) -> str:
    """
    Returns the coolest community of programming languages
    """
    return "Rust Community"

In [7]:
@tool
def currentTime(text: str) -> str:
    """
    Return current time 
    """
    return str(datetime.now())

In [11]:
class ChatBot:
    def __init__(self) -> None:
        self.callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        self.model = ChatOpenAI(api_key=api_key, model="gpt-3.5-turbo", max_tokens=500)
        self.tools = load_tools(['wikipedia'], llm=self.model)
        self.add_tool = self.tools + [coolest_community] + [currentTime]
        self.agents = initialize_agent(
            self.add_tool,
            self.model,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            handle_parsing_errors=True,
            verbose=True
        )
        
    def Chat(self, query: str) -> str:
        return self.agents(query)['output']

In [12]:
bot = ChatBot()

In [13]:
bot.Chat("Which is best community in programing languages")



> Entering new AgentExecutor chain...
I should use the "coolest_community" tool to find the best community in programming languages.
Action: coolest_community
Action Input: None
Observation: Rust Community
Thought:I now know the final answer
Final Answer: The Rust community is considered the best community in programming languages.

> Finished chain.


'The Rust community is considered the best community in programming languages.'

In [18]:
bot.Chat("Can you tell me a time")



> Entering new AgentExecutor chain...
I should use the currentTime tool to get the current time.
Action: currentTime
Action Input: None
Observation: 2024-02-04 00:28:03.121945
Thought:I now know the final answer
Final Answer: The current time is 2024-02-04 00:28:03.121945

> Finished chain.


'The current time is 2024-02-04 00:28:03.121945'